In [507]:
import io

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import bs4
import requests
import os
import csv
import re
from parse import *
import signal
import csv
import re

In [4]:
def parse_html(url):
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'lxml')
    return soup

In [135]:
url = 'https://pytorch.org/docs/stable/'
offset_list = []

In [491]:
soup = parse_html(url)
content = soup.find_all(class_='toctree-wrapper compound')
for c in content[2].find_all(class_='toctree-l1'):
    offset_list.append(c('a')[0].get('href'))

IndexError: list index out of range

In [47]:
offset_list

['torch.html',
 'nn.html',
 'nn.functional.html',
 'tensors.html',
 'tensor_attributes.html',
 'tensor_view.html',
 'autograd.html',
 'cuda.html',
 'amp.html',
 'distributed.html',
 'distributions.html',
 'hub.html',
 'jit.html',
 'nn.init.html',
 'onnx.html',
 'optim.html',
 'quantization.html',
 'rpc/index.html',
 'random.html',
 'sparse.html',
 'storage.html',
 'bottleneck.html',
 'checkpoint.html',
 'cpp_extension.html',
 'data.html',
 'dlpack.html',
 'model_zoo.html',
 'tensorboard.html',
 'type_info.html',
 'named_tensor.html',
 'name_inference.html',
 '__config__.html']

In [209]:
def parse_descp(func):
    s= re.sub('\n', ' ', str(func))
    descp_pat = r'<dl class=.*?>\s*<dt class=.*?>Parameters</dt>\s*<dd class=.*?>(.*?)</dd>'
    descp_sect = re.search(descp_pat, s)
    if not descp_sect:
        print('No parameter section detected')
        return None
    
    return descp_sect.group(1)
    

In [401]:
def get_param_sect(func_src):
    func_str= re.sub('\n', ' ', str(func_src))
    descp_pat = r'<dl class=.*?>\s*<dt class=.*?>Parameters</dt>\s*<dd class=.*?>(.*?)</dd>'
    descp_sect = re.search(descp_pat, func_str)
    if not descp_sect:
        print('no parameter section detected')

    return descp_sect.group(1)

In [435]:
def get_param_sect(func_src):
    func_str= re.sub('\n', ' ', str(func_src))
    print(func_str)
    descp_pat = r'<dl class=.*?>\s*<dt class=.*?>Parameters</dt>\s*<dd class=.*?>(.*?)</dd>'
    descp_sect = re.search(descp_pat, func_str)
    if not descp_sect:
        print('not detected')
        return

    return descp_sect.group(1)

In [436]:
get_param_sect(api_func[48])

<dl class="function"> <dt> <code class="sig-prename descclassname">torch.</code><code class="sig-name descname">where</code><span class="sig-paren">(</span><em class="sig-param">condition</em>, <em class="sig-param">x</em>, <em class="sig-param">y</em><span class="sig-paren">)</span> → Tensor</dt> <dd></dd></dl>
not detected


In [576]:

def get_sig(func_src):
    sig_pat = r'<span class=\"sig-paren\">\(</span>(.*?)<span class=\"sig-paren\">\)</span>'
    parsed = re.search(sig_pat, str(func_src))
    if parsed:
        sig = parsed.group(1)
        sig = re.split('\) -&gt;', sig, 1)[0]
        return sig
    else:
        print('error')

In [393]:
def get_title(func_src):
    sig_sect = func_src('dt')[0]
    param_pat = r'<span class=\"sig-paren\">\(</span>(.*?)<span class=\"sig-paren\">\)</span>'
    
    pat = r'<code class=\"sig-.*?\">(.*?)</code>'
    title = re.findall(pat, str(sig_sect))
    ret = ''
    for t in title:
        ret +=t
    return ret

In [181]:
def replace_symbol(descp, replace_dict):
    
    for s in replace_dict:
        descp = re.sub(s, replace_dict[s], descp)
    return descp
def parse_input(sig, replace_dict=None, detect_keywordonly=False):
    # parse signature
    # input_str: obj, allow_tensor=True, allow_operation=True  
    # need to take care of default value with comma, e.g. shape = [2,2]
    
    keywordonly = False
    keywordonly_args = []
    
    stripWS = lambda txt:'\''.join( it if i%2 else ''.join(it.split())  # remove space that is not qouted
        for i,it in enumerate(txt.split('\''))  )
    
    if replace_dict:
        sig = replace_symbol(sig, replace_dict)
    
    # sig = stripWS(sig)
    #parsed_str = re.split(',(?![0-9\(\)\[\]\{\}\'])', sig)  #str(input_str).split(',')
    parsed_str = re.split(r',\s*(?=[a-zA-Z\*])(?=[^)]*(?:\(|$))', sig) # parse by comma (,) that is not in brackets
    ret = {}

    for ps in parsed_str:
        ps = re.sub('[\']', '', ps)
        
        if ps =='*':
            keywordonly = True
            continue
            
        if '=' in ps:
            curr_arg, default = ps.split('=',1)
            ret[curr_arg] = default
            ps = curr_arg

        else:
            # no default value
            ret[ps] = None
        if detect_keywordonly and keywordonly:
            keywordonly_args.append(ps)

    return ret, keywordonly_args

In [586]:
BeautifulSoup(all_func[1], "html.parser")('dt')[0]

<dt><code class="sig-prename descclassname">torch.</code><code class="sig-name descname">where</code><span class="sig-paren">(</span><em class="sig-param">condition</em>, <em class="sig-param">x</em>, <em class="sig-param">y</em><span class="sig-paren">)</span> → Tensor</dt>

In [622]:
def get_ret_type(func):
    replace_dict = {
        r'</*(em|span).*?>':'',

    }
    sig_sect = func('dt')[0]

    # try to find -&gt;
    sig_pat = r'(-&gt;|→)(.*?)<(a|code|/dt).*?>'
    parsed = re.search(sig_pat, str(sig_sect))
    ret = ''
    if parsed:
        sig = parsed.group(2)

        ret = replace_symbol(sig, replace_dict)
        
        

    return ret

In [634]:
soup2 = parse_html('https://pytorch.org/docs/stable/cuda.html')#(url+offset_list[3])
api_func = soup2.find_all('dl',{'class':'function'})
api_class = soup2.find_all('dl',{'class':'class'})

In [635]:

param_pat = r'<span class=\"sig-paren\">\(</span>(.*?)<span class=\"sig-paren\">\)</span>'
for i, f in enumerate(api_func):
    name = get_title(f)
    raw_param = re.search(param_pat, str(f))
    print(i)
    print(name)
    print(get_ret_type(f))
    #print(f('dt')[0])
#     print(parse_input(raw_param.group(1), {r'</*em.*?>': '' }, detect_keywordonly=True))
# #     print(parse_descp(f))

0
torch.cuda.current_blas_handle

1
torch.cuda.current_device

2
torch.cuda.current_stream

3
torch.cuda.default_stream

4
torch.cuda.device_count

5
torch.cuda.get_device_capability

6
torch.cuda.get_device_name

7
torch.cuda.init

8
torch.cuda.ipc_collect

9
torch.cuda.is_available

10
torch.cuda.is_initialized

11
torch.cuda.set_device

12
torch.cuda.stream

13
torch.cuda.synchronize

14
torch.cuda.get_rng_state

15
torch.cuda.get_rng_state_all

16
torch.cuda.set_rng_state

17
torch.cuda.set_rng_state_all

18
torch.cuda.manual_seed

19
torch.cuda.manual_seed_all

20
torch.cuda.seed

21
torch.cuda.seed_all

22
torch.cuda.initial_seed

23
torch.cuda.comm.broadcast

24
torch.cuda.comm.broadcast_coalesced

25
torch.cuda.comm.reduce_add

26
torch.cuda.comm.scatter

27
torch.cuda.comm.gather

28
torch.cuda.empty_cache

29
torch.cuda.memory_stats

30
torch.cuda.memory_summary

31
torch.cuda.memory_snapshot

32
torch.cuda.memory_allocated

33
torch.cuda.max_memory_allocated

34
torch.cuda.r

In [639]:
api_func[37]

<dl class="function">
<dt id="torch.cuda.memory_cached">
<code class="sig-prename descclassname">torch.cuda.</code><code class="sig-name descname">memory_cached</code><span class="sig-paren">(</span><em class="sig-param">device=None</em><span class="sig-paren">)</span><a class="reference internal" href="_modules/torch/cuda/memory.html#memory_cached"><span class="viewcode-link">[source]</span></a><a class="headerlink" href="#torch.cuda.memory_cached" title="Permalink to this definition">¶</a></dt>
<dd><p>Deprecated; see <a class="reference internal" href="#torch.cuda.memory_reserved" title="torch.cuda.memory_reserved"><code class="xref py py-func docutils literal notranslate"><span class="pre">memory_reserved()</span></code></a>.</p>
</dd></dl>

In [132]:
get_title(api_func[1])

'bernoulli_'

In [161]:
total = []
for offset in offset_list:
    soup2 = parse_html(url+offset)
    
    total += result
    result = []


    api_func = soup2.find_all('dl',{'class':'function'})
    api_class = soup2.find_all('dl',{'class':'class'})


    for func in api_func:
        if len(func('dt'))>=1 and func('dt')[0].get('id')!=None:
            func_name = func('dt')[0].get('id')
        else:
            func_name = []
            for fn in func('code'):
                if fn.get('class')[0].startswith('sig-'):
                    func_name.append(fn.contents[0])
            func_name= ''.join(func_name)
        #print(func_name)

        # get parameters
        input_para = []
        paras = func.find_all(class_='sig-param')
        for p in paras:
            input_para.append(p.contents[0])

        # get href
        href = func.find_all(class_='headerlink')
        if len(href)>1:
            print(href)
        href = url+offset+href[0].get('href') if href!=[] else ''

        # get source code link if any
        source_code = ''
        try:
            for ri in func.find_all(class_='reference internal'):
                if ri('span')[0].get('class')[0] == 'viewcode-link':
                    source_code = url+ri.get('href')
                    break
        except:
            pass

        result.append(['', 'function', func_name, ','.join(input_para), href, source_code])
        
    for cls in api_class:
        cls_name = cls('dt')[0].get('id')
        # add an row of class
        
        # get href
        href = cls('dt')[0].find_all(class_='headerlink')
        if len(href)>1:
            print(href)
        href = url+offset+href[0].get('href') if href!=[] else ''
        
        # get source code link if any
        source_code = ''
        try:
            for ri in cls('dt')[0].find_all(class_='reference internal'):
                if ri('span')[0].get('class')[0] == 'viewcode-link':
                    source_code = url+ri.get('href')
                    break
        except:
            pass
        
        # input parameters
        input_para = []
        paras = cls('dt')[0].find_all(class_='sig-param')
        for p in paras:
            input_para.append(p.contents[0])
        
        result.append([cls_name, 'class', '', ','.join(input_para), href, source_code])
        
            
        for acdl in cls('dl'):

            if acdl.get('class') != None and acdl.get('class')[0] in ['method', 'attribute'] :
                func_name = acdl.find_all(class_='sig-name descname')
                href = acdl.find_all(class_='headerlink')

                # get input parameters
                paras = acdl.find_all(class_='sig-param')
                input_para = []
                for p in paras:
                    input_para.append(p.contents[0])


#                 if len(func_name)>1:
#                     print(func_name)

#                 if len(href)>1:
#                     print(href)
                #print(func_name)
                func_name = func_name[0].contents[0] if func_name!=[] else ''
                href = url+offset+href[0].get('href') if href!=[] else ''

                source_code = ''
                try:
                    for ri in acdl.find_all(class_='reference internal'):
                        if ri('span')[0].get('class')[0] == 'viewcode-link':
                            source_code = url+ri.get('href')
                            break
                except:
                    pass

                result.append([cls_name, acdl.get('class')[0], func_name, ','.join(input_para), href, source_code])
                #print(func_name+'\t'+str(input_para)+'\t'+href+'\t'+source_code )
    
    file_save = '/Users/danning/Desktop/deepflaw/DeepFlaw-Code/doc_analysis/pytorch/pytorch_pydoc_page/result/py_{}.csv'.format(offset[:-5])
    with open(file_save, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerow(['class', 'Type', 'name', 'input', 'link', 'source_code'])
        for r in result:
            csv_writer.writerow(r)
        print('{} saved, with {} rows'.format(file_save, len(result)))
        
file_save = '/Users/danning/Desktop/deepflaw/DeepFlaw-Code/doc_analysis/pytorch/result/py_total.csv'.format(offset[:-5])
with open(file_save, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(['class', 'Type', 'name', 'input', 'link', 'source_code'])
    for r in total:
        csv_writer.writerow(r)
    print('{} saved, with {} rows'.format(file_save, len(total)))

/Users/danning/Desktop/deepflaw/DeepFlaw-Code/doc_analysis/pytorch/result/py_torch.csv saved, with 287 rows
/Users/danning/Desktop/deepflaw/DeepFlaw-Code/doc_analysis/pytorch/result/py_nn.csv saved, with 257 rows
/Users/danning/Desktop/deepflaw/DeepFlaw-Code/doc_analysis/pytorch/result/py_nn.functional.csv saved, with 99 rows
/Users/danning/Desktop/deepflaw/DeepFlaw-Code/doc_analysis/pytorch/result/py_tensors.csv saved, with 357 rows
/Users/danning/Desktop/deepflaw/DeepFlaw-Code/doc_analysis/pytorch/result/py_tensor_attributes.csv saved, with 3 rows
/Users/danning/Desktop/deepflaw/DeepFlaw-Code/doc_analysis/pytorch/result/py_autograd.csv saved, with 34 rows
/Users/danning/Desktop/deepflaw/DeepFlaw-Code/doc_analysis/pytorch/result/py_cuda.csv saved, with 59 rows
/Users/danning/Desktop/deepflaw/DeepFlaw-Code/doc_analysis/pytorch/result/py_distributed.csv saved, with 26 rows
/Users/danning/Desktop/deepflaw/DeepFlaw-Code/doc_analysis/pytorch/result/py_distributions.csv saved, with 388 rows